In [1]:
import os
import urllib
import numpy as np
import pandas as pd
from datetime import datetime,timedelta

In [2]:
today = datetime.now().strftime('%Y-%m-%d')
yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')

In [3]:
def QAQC_datapull(a,b,c,d):
    if a == 'hourly':
        e = ['tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir']
        r = pd.date_range(start=c,end=d,freq='H')
    elif a == 'five_minute':
        e = ['tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir']
        r = pd.date_range(start=c,end=d,freq='5min')
    elif a == 'daily':
        e = ['tave,tmax,tmin,vp,rhmax,rhmin,sr,wrun,pp,st5mx,st5mn,st15mx,st15mn,gust,gustdir']
        r = pd.date_range(start=c,end=d,freq='D')
        
    urllib.request.urlretrieve('http://coagmet.colostate.edu/cgi-bin/web_services.pl?' +
                              'type=' + a +
                              '&sids=' + b +
                              '&sdate=' + c +
                              '&edate=' + d +
                              '&elems=' + e,
                              filename=csv)
    
    data = pd.read_csv(csv)
    
    data = data.reset_index()
    
    headers = e.split(',')
    
    headers.insert(0,'date')
    headers.instert(0,'station')
    
    data.columns = headers
    
    data['date'] = pd.to_datetime(data.date)
    
    data = data.set_index('date').reindex(r,copy=False).rename_axis('date')
    
    return data

In [5]:
def fivemin_report(x):
    file = (x['path'] + r'\fivemin\QAQC_report_' + x['drainage'] + 
            '_' + x['region'] + "_" + yesterday + '.txt')
    f = open(file,'a')
    title = (yesterday + r' QA/QC Report for stations in the ' + x['region'] +
            ' region of the ' + x['drainage'] + ' drainage\n')
    
    f.write(title)
    f.write('/n')
    
    n = 0
    data = QAQC_datapull('five_minute',x['stations'],yesterday,today)
    
    station = data['station'][n]
    date = data['date'][n]
    
    f.write('Five Minute Data/n')
    f.write('/n')
    f.write('Missing Data:/n')
    f.write('_____________/n')
    
    n = 0
    for index,rows in data.iterrows():
        if(pd.isnull(data['tmean'][n])):
            f.write(station + 
                    ' is missing temperature data at ' + date + '/n')
        elif(pd.isnull(data['rh'][n])):
            f.write(station + 
                    ' is missing relative humidity data at ' + date + '/n')
        elif(pd.isnull(data['vp'][n])):
            f.write(station + ' is missing vapor pressure data at ' + date + '/n')
        elif(pd.isnull(data['sr'][n])):
            f.write(station + 
                    ' is missing solar radiation data at ' + date + '/n')
        elif(pd.isnull(data['ws'][n])):
            f.write(station + 
                   ' is missing wind speed data at ' + date + '/n')
        elif(pd.isnull(data['wind_vec'][n])):
            f.write(station + ' is missing wind direction data at ' + date + '/n')
        elif(pd.isnull(data['wind_std'][n])):
            f.write(station + 
                   ' is missing wind direction standard deviation data at ' + date + '/n')
        elif(pd.isnull(data['pp'][n])):
            f.write(station + 
                   ' is missing precipitation data at ' + date + '/n')
        elif(pd.isnull(data['st5'][n])):
            f.write(station + 
                   ' is missing 5cm soil temperature data at ' + date + '/n')
        elif(pd.isnull(data['st15'][n])):
            f.write(station + 
                   ' is missing 15cm soil temperature data at ' + date + '/n')
        elif(pd.isnull(data['gust'][n])):
            f.write(station + ' is missing gust speed data at ' + data + '/n')
        elif(pd.isnull(data['gustdir'][n])):
            f.write(station + ' is missing gust direction data at ' + date + '/n')
        n = n + 1
    f.write('/n')
    f.write('/n')
        
    f.write('Temperature Data Report/n')
    f.write('_______________________/n')
    
                    
    n = 0
    for index,rows in data.iterrows():
        if data['tmean'][n] > 46.1:
            f.write(station + ' reported a temperature of ' + str(data['tmean'][n]) + 
                    '°C on ' + date + '. If true, this a new state record for max temp./n')
        elif data['tmean'][n] < -51.7:
            f.write(station + ' reported a temperature of ' + str(data['tmean'][n]) +
                   '°C on ' + date + '. If true, this a new state record for min temp./n')
        n = n + 1 
    f.write('/n')
    f.write('/n')
    
    f.write('Relative Humidity Data Report/n')
    f.write('_____________________________/n')
    
    n = 0
    for index,rows in data.iterrows():
        if data['rh'][n] > 1.0:
            f.write(station + ' reported a relative humidity of ' + str(data['rh'][n]) +
                    ' at ' + date + '/n')
        elif data['rh'][n] < 0.0:
            f.write(station + ' reported negative relative humidity at ' +
                   date + '/n')
        elif data['rh'][n] < 0.01:
            f.write(date + ' broke the world record for low relative humidity at ' + 
                   station + '/n')
        n = n + 1
        
    f.write('/n')
    f.write('/n')
    
    f.write('Vapor Pressure Data Report/n')
    f.write('__________________________/n')
    #
    #
    #
    #
    
    f.write('/n')
    f.write('/n')
    
    f.write('Solar Radiation Data Report/n')
    f.write('______________________/n')
    n = 0
    for index,rows in data.iterrows():
        if data['sr'][n] < 0.0:
            f.write(station + ' reported negative solar radiation at ' + date + '/n')
        n = n + 1
    
    f.write('/n')
    f.write('/n')
    
    f.write('Wind Speed Data Report/n')
    f.write('_________________/n')
    
    n = 0
    for index,rows in data.iterrows():
        if data['ws'][n] < 0.0:
            f.write(station + ' reported negative wind speed at ' + date + '/n')
        n = n + 1
        
    f.write('/n')
    f.write('/n')
    
    f.write('Wind Direction')
    
    n = 0 
    for index,rows in data.iterrows():
        if data['wind_vec'][n] < 0.0:
            f.write(station + ' reported negative wind direction at ' + date + '/n')
    
    f.write('/n')
    f.write('/n')
    